In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
%cd '/content/drive/My Drive/UF/Intro to Data Science/Project/Modelling'

/content/drive/My Drive/UF/Intro to Data Science/Project/Modelling
/content/drive/My Drive/UF/Intro to Data Science/Project/Modelling


In [6]:
%ls

11152019_test_final.csv  11152019_train_final.csv  linear_clf.pickle
11152019_test_final.zip  11152019_train_final.zip  modelling_corr.ipynb
11152019_test_final.csv  11152019_train_final.csv  linear_clf.pickle
11152019_test_final.zip  11152019_train_final.zip  modelling_corr.ipynb


In [0]:
import numpy as np
import pandas as pd
import sklearn.svm as svm
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
import pickle

In [8]:
train = pd.read_csv("11152019_train_final.csv")
test = pd.read_csv("11152019_test_final.csv")
train.shape, test.shape

((1537503, 42), (361429, 42))

In [9]:
train.columns

Index(['channelGrouping', 'device.browser', 'device.deviceCategory',
       'device.isMobile', 'device.operatingSystem', 'fullVisitorId',
       'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country',
       'geoNetwork.networkDomain', 'geoNetwork.region',
       'geoNetwork.subContinent', 'totals.bounces', 'totals.hits',
       'totals.newVisits', 'totals.pageviews', 'totals.sessionQualityDim',
       'totals.timeOnSite', 'totals.totalTransactionRevenue',
       'totals.transactionRevenue', 'totals.transactions',
       'trafficSource.adContent',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign',
       'trafficSource.isTrueDirect', 'trafficSource.keyword',
       'trafficSource.medium', 'trafficSource.referralPath',
       'trafficSource.source', 'visitId', 'visitNumber', 'visitStartTime',
       'new.returni

In [10]:
train.isna().sum(axis=0)

channelGrouping                                 0
device.browser                                  0
device.deviceCategory                           0
device.isMobile                                 0
device.operatingSystem                          0
fullVisitorId                                   0
geoNetwork.city                                 0
geoNetwork.continent                            0
geoNetwork.country                              0
geoNetwork.networkDomain                        0
geoNetwork.region                               0
geoNetwork.subContinent                         0
totals.bounces                                  0
totals.hits                                     0
totals.newVisits                                0
totals.pageviews                                0
totals.sessionQualityDim                        0
totals.timeOnSite                               0
totals.totalTransactionRevenue                  0
totals.transactionRevenue                       0


In [0]:
def preprocess(df):

  y = df['new.returningCustomer']
  X = df.drop(columns=['new.returningCustomer'])
  keep_cols = ['totals.bounces',
              'totals.hits',
              'totals.newVisits',
              'totals.pageviews',
              'totals.sessionQualityDim',
              'totals.timeOnSite',
              'trafficSource.isTrueDirect',
              'visitNumber',
              'country_dem',
              'latitude',
              'longitude']

  X = X[keep_cols]

  return X,y

def encode(X):

  lb = preprocessing.LabelBinarizer()
  X['trafficSource.isTrueDirect'] = lb.fit_transform(X['trafficSource.isTrueDirect'])

  # Not in keep_cols
  # X['device.isMobile'] = lb.transform(X['device.isMobile'])

  # Not in keep_cols
  # X['visitStartTime'].dtype == np.int64

  ignoreColumns = {''}
  for column in X.columns:
      if (X[column].dtype != np.int64 and X[column].dtype != np.float64):
          print(f"Converting column: {column}")

          le = preprocessing.LabelEncoder()
          X[column] = le.fit_transform(X[column])
          del le

  return X

def scale(X):
  scaler = preprocessing.StandardScaler()
  X[X.columns] = scaler.fit_transform(X[X.columns])
  return X

In [0]:
X, y = preprocess(train)
X = encode(X)
X = scale(X)

In [17]:
X.shape, y.shape

((1537503, 11), (1537503,))

In [18]:
linear_svc = svm.LinearSVC(verbose=1)
linear_svc

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=1)

In [19]:
X_test, y_test = preprocess(test)
X_test.head()

,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,trafficSource.isTrueDirect,visitNumber,country_dem,latitude,longitude
0,-1.0,4,0.0,3.0,1,973.0,True,2,26043,20.593684,78.962880
1,-1.0,4,0.0,3.0,1,49.0,True,166,162791,37.090240,-95.712891
2,-1.0,4,0.0,3.0,1,24.0,True,2,162791,37.090240,-95.712891
3,-1.0,5,0.0,4.0,1,25.0,True,4,162791,37.090240,-95.712891
4,-1.0,5,1.0,4.0,1,49.0,False,1,162791,37.090240,-95.712891


In [20]:
X_test = encode(X_test)
X_test.dtypes

totals.bounces                float64
totals.hits                     int64
totals.newVisits              float64
totals.pageviews              float64
totals.sessionQualityDim        int64
totals.timeOnSite             float64
trafficSource.isTrueDirect      int64
visitNumber                     int64
country_dem                     int64
latitude                      float64
longitude                     float64
dtype: object

In [21]:
X_test = scale(X_test)
X_test.describe()

,totals.bounces,totals.hits,totals.newVisits,totals.pageviews,totals.sessionQualityDim,totals.timeOnSite,trafficSource.isTrueDirect,visitNumber,country_dem,latitude,longitude
count,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05,3.614290e+05
mean,8.440292e-16,-3.907622e-15,-2.566713e-15,-8.597852e-15,-7.528747e-15,2.846723e-15,-1.193545e-14,-1.636408e-15,3.051309e-14,-9.451451e-15,1.187592e-14
std,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00,1.000001e+00
min,-9.139619e-01,-4.559527e-01,-1.573631e+00,-4.954160e-01,-2.977721e-01,-3.847782e-01,-7.659207e-01,-1.385967e-01,-1.013661e+00,-3.998602e+00,-1.747320e+00
25%,-9.139619e-01,-4.559527e-01,-1.573631e+00,-4.954160e-01,-2.977721e-01,-3.847782e-01,-7.659207e-01,-1.385967e-01,-9.488859e-01,-3.342659e-01,-8.531999e-01
50%,-9.139619e-01,-3.481594e-01,6.354729e-01,-3.414586e-01,-2.977721e-01,-3.586454e-01,-7.659207e-01,-1.385967e-01,-6.753372e-01,2.214188e-01,-6.045244e-01
75%,1.094137e+00,8.301387e-02,6.354729e-01,1.204136e-01,-2.977721e-01,-1.077706e-01,1.305618e+00,-4.550683e-02,1.101223e+00,2.214188e-01,6.668346e-01
max,1.094137e+00,5.333291e+01,6.354729e-01,7.632933e+01,7.250969e+00,4.175174e+01,1.305618e+00,4.845431e+01,1.101223e+00,2.094501e+00,2.452350e+00


In [0]:
gaussianNB = GaussianNB()
# gaussianNB.fit(X, y)

In [23]:
logistic_clf = LogisticRegression()
logistic_clf

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
rf_clf = RandomForestClassifier(random_state=0)

In [25]:
eclf1 = VotingClassifier(estimators=[('lsvc', linear_svc), ('gnb', gaussianNB), ('log', logistic_clf), ('rf', rf_clf)], voting='hard')
eclf1 = eclf1.fit(X, y)

[LibLinear]

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [26]:
eclf1.score(X, y)

0.8888327372369355

In [27]:
eclf1.score(X_test, y_test)

0.863740873034538